In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True)
cdf.printSchema()
cdf.show(3)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
+--------+------+-------------+--------+-----------+-------------+
only showing top 3 rows



In [4]:
type(cdf)

pyspark.sql.dataframe.DataFrame

In [5]:
cdf.columns

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']

### 각 반 학생수(class_std_cnt)가 평균 반 학생수보다 많은 class의 data를 추출하시오

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']
반코드        학교이름   반 학생수       지역    학교형태       교육형태

In [6]:
# view 생성
cdf.createOrReplaceTempView('class')

In [ ]:
# spark.sql(
    # spark.sql.query문자열
)
# spark가 쿼리 문자열을 분석(파싱)하고 분석된 결과로 연산계획을 생성

In [7]:
cdf.show()

[Stage 2:>                                                          (0 + 1) / 1]

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
|     EPS| CCAAW|           20|Suburban| Non-public| Experimental|
|     IQN| CCAAW|           15|Suburban| Non-public| Experimental|
|     PGK| CCAAW|           21|Suburban| Non-public|     Standard|
|     UHU| CCAAW|           16|Suburban| Non-public| Experimental|
|     UWK| CCAAW|           19|Suburban| Non-public|     Standard|
|     A33| CIMBB|           19|   Urban| Non-public|     Standard|
|     EID| CIMBB|           21|   Urban| Non-public|     Standard|
|     HUJ| CIMBB|           17|   Urban| Non-public| Experimental|
|     PC6| CIMBB|           17|   Urban| Non-public|     Stand

In [9]:
# where 절에서 평균 반 학생수를 구하고 그 값보다 많은 반을 추출
# 에러발생 : where절에서는 집계함수 사용 불가
# spark.sql('''
#     select * from class
#     where class_std_cnt >= avg(class_std_cnt) and class_std_cnt is not null
# ''')

- subquery 사용 : ()로 우선순위 높여야 함
    - where절에서 사용하는 예제
  1. subquery 이용 평균 반 학생수를 전달하기
  2. where 절에서 반 학생수가 1에서 전달받은 값보다 큰 data 추출

In [11]:
# school컬럼과 avg(class_std_cnt) 컬럼의 원소수의 차이가 있음 -> df 구성 불가능
# spark.sql('select school, avg(class_std_cnt) from class where class_std_cnt is not null').show()

In [12]:
# 1. subquery - 평균 반 학생수 select 하는 쿼리(select~from~where 구조일때 집계함수는 select 절에만 사용가능)
spark.sql('select avg(class_std_cnt) from class where class_std_cnt is not null').show()

+------------------+
|avg(class_std_cnt)|
+------------------+
|21.828282828282827|
+------------------+



In [12]:
#2. 1의 query를 where에 비교 대상으로 사용(subquery 적용)
spark.sql('''
    select * from class
    where class_std_cnt >= (select avg(class_std_cnt) from class) and (class_std_cnt is not null)
''')

DataFrame[class_cd: string, school: string, class_std_cnt: string, loc: string, school_type: string, teaching_type: string]

### DF method  사용

In [11]:
cdf.select('*')\
   .where(cdf.class_std_cnt >= avg(cdf.class_std_cnt))
# where절에서 집계함수 사용은 가능하지만
# 단, 집계함수가 반환하는 타입이 값이 아닌 row객체 반환
# 객체와 값 비교연산 에러 발생, avg(cdf.class_std_cnt)이 코드가 객체 아닌 값으로 명시가 되어야 함 - 연산결과 추출 후 값만 다시 추출

In [15]:
type(cdf.select(avg('class_std_cnt'))) # df
cdf.select(avg('class_std_cnt')).collect() # 연산결과 반환 - 파이썬 리스트로 반환
cdf.select(avg('class_std_cnt')).collect()[0]
cdf.select(avg('class_std_cnt')).collect()[0][0]


pyspark.sql.dataframe.DataFrame

[Row(avg(class_std_cnt)=21.828282828282827)]

Row(avg(class_std_cnt)=21.828282828282827)

21.828282828282827

In [17]:
# df method 사용
cdf.select('*')\
   .where(cdf.class_std_cnt >= cdf.select(avg('class_std_cnt')).collect()[0][0]).show(2)

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     1Q1| CUQAM|           28|Urban|     Public|     Standard|
|     BFY| CUQAM|           27|Urban|     Public|     Standard|
+--------+------+-------------+-----+-----------+-------------+
only showing top 2 rows



### 복잡한 연산도 해보자

#### case 0. 학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자
- 단, 지역정보가 없는 데이터는 제외한다

In [20]:
cdf.show(10)

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
|     EPS| CCAAW|           20|Suburban| Non-public| Experimental|
|     IQN| CCAAW|           15|Suburban| Non-public| Experimental|
|     PGK| CCAAW|           21|Suburban| Non-public|     Standard|
|     UHU| CCAAW|           16|Suburban| Non-public| Experimental|
|     UWK| CCAAW|           19|Suburban| Non-public|     Standard|
|     A33| CIMBB|           19|   Urban| Non-public|     Standard|
|     EID| CIMBB|           21|   Urban| Non-public|     Standard|
+--------+------+-------------+--------+-----------+-------------+
only showing top 10 rows



- DF 모듈 활용

#### count('컬럼')
- 중복을 포함해서 개수를 셈
#### count_distinct('컬럼')  
- 중복을 제외하고 개수를 셈

In [24]:
cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(count('school').alias('school_cnt')).show()

+--------+----------+
|     loc|school_cnt|
+--------+----------+
|   Urban|        37|
|Suburban|        34|
|   Rural|        26|
+--------+----------+



In [22]:
cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(count_distinct('school').alias('school_cnt')).show()

+--------+----------+
|     loc|school_cnt|
+--------+----------+
|   Urban|         9|
|Suburban|         7|
|   Rural|         7|
+--------+----------+



In [19]:
# 학교가 가장 많이 위치한 지역의 학교수와 가장 적게 위치한 지역의 학교수 구해 리스트로 반환
cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(count('school').alias('school_cnt'))\
    .agg(max('school_cnt').alias('maxCnt'), min('school_cnt').alias('minCnt'))\
    .show()
          

+------+------+
|maxCnt|minCnt|
+------+------+
|    37|    26|
+------+------+



In [21]:
# 학교의 중복 제외후 count
cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(count_distinct('school').alias('school_cnt'))\
    .agg(max('school_cnt').alias('maxCnt'), min('school_cnt').alias('minCnt'))\
    .show()

+------+------+
|maxCnt|minCnt|
+------+------+
|     9|     7|
+------+------+



In [59]:
[[9],[7]]

[[9], [7]]

In [37]:
# 아래 코드 action 진행 후 결과 반환
cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(count_distinct('school').alias('school_cnt'))\
    .agg(max('school_cnt').alias('maxCnt'), min('school_cnt').alias('minCnt'))\
    .collect()[0]

cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(count_distinct('school').alias('school_cnt'))\
    .agg(max('school_cnt').alias('maxCnt'), min('school_cnt').alias('minCnt'))\
    .collect()[0][0]

cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(count_distinct('school').alias('school_cnt'))\
    .agg(max('school_cnt').alias('maxCnt'), min('school_cnt').alias('minCnt'))\
    .collect()[0][1]

Row(maxCnt=9, minCnt=7)

9

7

In [40]:
[
  cnt for cnt  in
            cdf.where(cdf.loc.isNotNull())\
            .groupby(cdf.loc)\
            .agg(count_distinct('school').alias('school_cnt'))\
            .agg(max('school_cnt').alias('maxCnt'), min('school_cnt').alias('minCnt'))\
            .collect()[0]
]  
# 1차원 리스트

[9, 7]

- column객체.isin([])
    - column객체의 값이 함수 내부의 집합데이터가 포함하고 있는지를 확인해서 T/F반환하는 함수
    - isin함수내부 데이터는 list가 가장 일반적 - 1차원 list여야함

In [43]:
# 학교가 가장 많이 위치한 지역의 총학생수와 학교가 가장 적게 위치한 지역의 총 학생수 구하기
# 가장많거나 적은 지역이 2개이상이면 총학생수가 적은 지역을 학교수가 가장 적은 지역으로 결정, 많은 지역은 학생수가 많은 지역을 가장 많은 지역으로 결정
# 지역의 학교수가 9개 이거나 지역의 학교수가 7개인 지역의 지역별 총 학생수
cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where(count_distinct("school").isin(
            [ cnt for cnt in
                            cdf.where(cdf.loc.isNotNull())\
                            .groupby(cdf.loc)\
                            .agg(count_distinct('school').alias('school_cnt'))\
                            .agg(max('school_cnt').alias('maxCnt'), min('school_cnt').alias('minCnt'))\
                            .collect()[0]
            ]
            ))\
    .show()

+--------+-------+
|     loc|std_cnt|
+--------+-------+
|   Urban|  906.0|
|Suburban|  717.0|
|   Rural|  538.0|
+--------+-------+



In [44]:
print("학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이 구하기")
cdf.where(cdf.loc.isNotNull())\
    .groupby(cdf.loc)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where(count_distinct("school").isin(
            [ cnt for cnt in
                            cdf.where(cdf.loc.isNotNull())\
                            .groupby(cdf.loc)\
                            .agg(count_distinct('school').alias('school_cnt'))\
                            .agg(max('school_cnt').alias('maxCnt'), min('school_cnt').alias('minCnt'))\
                            .collect()[0]
            ]
            ))\
    .select((max('std_cnt') - min('std_cnt')).alias('학생수차이'))\
    .show()

학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이 구하기
+----------+
|학생수차이|
+----------+
|     368.0|
+----------+



- spark.sql

In [56]:
# 각 지역별 학교수, 단, 지역정보가 없는 레코드는 제외
# sql쿼리의 중복제거 distinct 컬럼명
spark.sql('''
    select count(distinct school) as `지역별학교수`
    from class
    where loc is not null
    group by loc
''').show()

+------------+
|지역별학교수|
+------------+
|           9|
|           7|
|           7|
+------------+



In [58]:
# from절에 subquery 사용
# 지역별 학교수 최대/최소값
spark.sql('''
    select max(cntSCH) as max, min(cntSCH) as min
    from (
        select count(distinct school) as cntSCH
        from class
        where loc is not null
        group by loc
    )
''').show()



+---+---+
|max|min|
+---+---+
|  9|  7|
+---+---+



#### df 다중열
- in 연산자를 사용하려면 다중열 비교 주체가 두개여야 비교 가능
- ex. 다중행 in : where school in ['abc','def','ghi']
- ex. 다중열 in : where school, class in [['abc'],['def']] - school컬럼값이 첫번째열(abc)에 있는지 확인, class 컬럼값이 두번째열(def)

In [64]:
## 학교가 가장 많이 위치한 지역의 총학생수와 학교가 가장 적게 위치한 지역의 총 학생수 구하기
## having절에 subquery
## having절은 groupby에 종속이어시 집계함수 사용가능
spark.sql('''
    select sum(class_std_cnt) as tot
    from class
    where loc is not null
    group by loc
    having count(distinct school)= (select max(cntSCH) as max
                                        from (
                                            select count(distinct school) as cntSCH
                                            from class
                                            where loc is not null
                                            group by loc
                                        )
                                    ) or
            count(distinct school)= (select min(cntSCH) as min
                                        from (
                                            select count(distinct school) as cntSCH
                                            from class
                                            where loc is not null
                                            group by loc
                                        )
                                    )
''').show()
# 결과 단일열 다중행

+-----+
|  tot|
+-----+
|906.0|
|717.0|
|538.0|
+-----+



In [66]:
print("학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이 구하기")
spark.sql ('''
    select (max(tot) - min(tot)) as `학생수차이`
    from (
            select sum(class_std_cnt) as tot
            from class
            where loc is not null
            group by loc
            having count(distinct school)= (select max(cntSCH) as max
                                                from (
                                                    select count(distinct school) as cntSCH
                                                    from class
                                                    where loc is not null
                                                    group by loc
                                                )
                                            ) or
                    count(distinct school)= (select min(cntSCH) as min
                                                from (
                                                    select count(distinct school) as cntSCH
                                                    from class
                                                    where loc is not null
                                                    group by loc
                                                )
                                            )
    )
''').show()

학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이 구하기
+----------+
|학생수차이|
+----------+
|     368.0|
+----------+



#### case 1. 소속된 반의 개수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교의 학생 수를 구해보자
#### 단, 학교가 null인 데이터는 제외한다

In [78]:
# from절 subquery : inline view
# 학급수가 3개 이상인 학교들의 총 학생수
spark.sql('''
    select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
    from classV
    where school is not null
    group by school
    having count(class_cd) >= 3
''').show(30)


+------+------+------+
|school|학급수|학생수|
+------+------+------+
| VHDHF|     3|  51.0|
| LAYPA|     3|  57.0|
| GOOBU|     6| 158.0|
| UUUQX|     5|  84.0|
| CIMBB|     4|  74.0|
| UKPGS|     6| 128.0|
| UAGPU|     4|  87.0|
| CCAAW|     6| 109.0|
| FBUMG|     3|  46.0|
| ZOWMK|     4| 117.0|
| ZMNYA|     3|  69.0|
| QOQTS|     6| 149.0|
| CUQAM|     4| 107.0|
| OJOBU|     4|  81.0|
| GOKXL|     3|  64.0|
| GJJHK|     5| 118.0|
| KZKKE|     5| 111.0|
| DNQDD|     5| 122.0|
| VKWQH|     5| 100.0|
| IDGFP|     5|  94.0|
| VVTVA|     4| 114.0|
+------+------+------+



In [87]:
#학급수가 3개 이상인 학교들의 총 학생수중 최소값
spark.sql('''
    select min(`학생수`) as `최소학생수`
    from (
        select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
        from classV
        where school is not null
        group by school
        having count(class_cd) >= 3
    )
''').show()

+----------+
|최소학생수|
+----------+
|      46.0|
+----------+



### DF method

In [82]:
#학급수가 3개 이상인 학교들의 총 학생수
cdf.groupby(cdf.school)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where((count(cdf.class_cd) >=3) & (cdf.school.isNotNull()))\
    .show()

+------+-------+
|school|std_cnt|
+------+-------+
| VHDHF|   51.0|
| LAYPA|   57.0|
| GOOBU|  158.0|
| UUUQX|   84.0|
| CIMBB|   74.0|
| UKPGS|  128.0|
| UAGPU|   87.0|
| CCAAW|  109.0|
| FBUMG|   46.0|
| ZOWMK|  117.0|
| ZMNYA|   69.0|
| QOQTS|  149.0|
| CUQAM|  107.0|
| OJOBU|   81.0|
| GOKXL|   64.0|
| GJJHK|  118.0|
| KZKKE|  111.0|
| DNQDD|  122.0|
| VKWQH|  100.0|
| IDGFP|   94.0|
+------+-------+
only showing top 20 rows



In [85]:
#학급수가 3개 이상인 학교들의 총 학생수중 최소값
cdf.groupby(cdf.school)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where((count(cdf.class_cd) >=3) & (cdf.school.isNotNull()))\
    .select(min('std_cnt').alias('최소학생수'))\
    .show()

+----------+
|최소학생수|
+----------+
|      46.0|
+----------+



#### case 1_1. 학급수가 3개 이상인 학교들 중 학생 숫자가 가장 적은 학교의 학생 수와 학교명을 추출하자
#### 단, 학교가 null인 데이터는 제외한다

### sql query

In [67]:
cdf.createOrReplaceTempView('classV')

In [73]:
# 학급수가 3개 이상인 학교들의 총 학생수
spark.sql('''
    select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
    from classV
    where school is not null
    group by school
    having count(class_cd) >= 3
''').show(30)

+------+------+------+
|school|학급수|학생수|
+------+------+------+
| VHDHF|     3|  51.0|
| LAYPA|     3|  57.0|
| GOOBU|     6| 158.0|
| UUUQX|     5|  84.0|
| CIMBB|     4|  74.0|
| UKPGS|     6| 128.0|
| UAGPU|     4|  87.0|
| CCAAW|     6| 109.0|
| FBUMG|     3|  46.0|
| ZOWMK|     4| 117.0|
| ZMNYA|     3|  69.0|
| QOQTS|     6| 149.0|
| CUQAM|     4| 107.0|
| OJOBU|     4|  81.0|
| GOKXL|     3|  64.0|
| GJJHK|     5| 118.0|
| KZKKE|     5| 111.0|
| DNQDD|     5| 122.0|
| VKWQH|     5| 100.0|
| IDGFP|     5|  94.0|
| VVTVA|     4| 114.0|
+------+------+------+



In [77]:
#학급수가 3개 이상인 학교들의 총 학생수중 최소값
spark.sql('''
    select min(`학생수`)
    from (
        select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
        from classV
        where school is not null
        group by school
        having count(class_cd) >= 3
    )
''').show()

+-----------+
|min(학생수)|
+-----------+
|       46.0|
+-----------+



In [92]:
#학급수가 3개 이상인 학교들의 총 학생수중 최소값인학교의 이름과 학생수
spark.sql('''
    select school as `최소학생수인 학교`, sum(class_std_cnt) as `학생수`
    from classV
    group by school
    having sum(class_std_cnt) == (select min(`학생수`)
                                    from (
                                        select school, count(class_cd) as `학급수`, sum(class_std_cnt) as `학생수`
                                        from classV
                                        where school is not null
                                        group by school
                                        having count(class_cd) >= 3))
    
''').show(50)

+-----------------+------+
|최소학생수인 학교|학생수|
+-----------------+------+
|            FBUMG|  46.0|
+-----------------+------+



### df moethod 사용

In [93]:
#학급수가 3개 이상인 학교들의 총 학생수
cdf.groupby(cdf.school)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where((count(cdf.class_cd) >=3) & (cdf.school.isNotNull()))\
    .select(min('std_cnt'))\
    .show()

+------------+
|min(std_cnt)|
+------------+
|        46.0|
+------------+



In [97]:
#
cdf.groupby(cdf.school)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where((count(cdf.class_cd) >=3) & (cdf.school.isNotNull()))\
    .select(min('std_cnt'))\
    .collect()[0][0]         

46.0

In [104]:
cdf.groupby(cdf.school)\
    .agg(sum('class_std_cnt').alias('std_cnt'))\
    .where(cdf.school.isNotNull())\
    .where(col('std_cnt') ==  cdf.groupby(cdf.school)\
                                .agg(sum('class_std_cnt').alias('std_cnt'))\
                                .where((count(cdf.class_cd) >=3) & (cdf.school.isNotNull()))\
                                .select(min('std_cnt'))\
                                .collect()[0][0]  
          ).show()

+------+-------+
|school|std_cnt|
+------+-------+
| FBUMG|   46.0|
+------+-------+



### case2. 1.지역에 따른 학교로 분류하고 분류된 학교의 class_cd가 2개 초과인 학교별로 각반의 학생수가 가장 작은 반의 학생수를 구하시오
### 위에서 구한 학생수중 가장 큰 값은 얼마인가?
1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출
2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출
3. 2번에서 추출된 학생수들 중 가장 큰 수 추출

### sql query 활용

In [ ]:
# 참고
# Urban 지역의 학교 VVTVA는 반수가 4개고 각 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



### sql query

In [107]:
# 1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출 - 지역에 따른 학교별 반 개수
spark.sql('''
    select loc, school, count(class_cd)
    from classV
    group by loc, school
    having count(class_cd) > 2
''').show(50)

+--------+------+---------------+
|     loc|school|count(class_cd)|
+--------+------+---------------+
|   Rural| VHDHF|              3|
|   Urban| GOOBU|              6|
|Suburban| ZMNYA|              3|
|   Urban| ZOWMK|              4|
|    NULL|  NULL|              3|
|Suburban| UUUQX|              5|
|Suburban| DNQDD|              5|
|   Urban| CUQAM|              4|
|   Urban| IDGFP|              5|
|   Rural| GOKXL|              3|
|   Rural| KZKKE|              5|
|   Rural| VKWQH|              5|
|Suburban| CCAAW|              6|
|   Rural| LAYPA|              3|
|   Urban| QOQTS|              6|
|Suburban| UAGPU|              4|
|   Rural| FBUMG|              3|
|   Urban| VVTVA|              4|
|Suburban| UKPGS|              6|
|Suburban| GJJHK|              5|
|   Rural| OJOBU|              4|
|   Urban| CIMBB|              4|
+--------+------+---------------+



In [128]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [110]:
# 2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출 단, school값이  null 인 데이터 제외
spark.sql('''
    select loc, school, count(class_cd) as cnt, min(class_std_cnt) as minStd
    from classV
    group by loc, school
    having count(class_cd) > 2 and school is not null
''').show(30)  

+--------+------+---+------+
|     loc|school|cnt|minStd|
+--------+------+---+------+
|   Rural| FBUMG|  3|    14|
|   Rural| GOKXL|  3|    19|
|   Rural| KZKKE|  5|    20|
|   Rural| LAYPA|  3|    17|
|   Rural| OJOBU|  4|    17|
|   Rural| VHDHF|  3|    15|
|   Rural| VKWQH|  5|    18|
|Suburban| CCAAW|  6|    15|
|Suburban| DNQDD|  5|    20|
|Suburban| GJJHK|  5|    21|
|Suburban| UAGPU|  4|    21|
|Suburban| UKPGS|  6|    18|
|Suburban| UUUQX|  5|    15|
|Suburban| ZMNYA|  3|    22|
|   Urban| CIMBB|  4|    17|
|   Urban| CUQAM|  4|    24|
|   Urban| GOOBU|  6|    24|
|   Urban| IDGFP|  5|    17|
|   Urban| QOQTS|  6|    22|
|   Urban| VVTVA|  4|    25|
|   Urban| ZOWMK|  4|    27|
+--------+------+---+------+



In [112]:
# 3. 2번에서 추출된 학생수들 중 가장 큰 수 추출
spark.sql('''
        select max(minStd) as maxMinStd
        from (
            select loc, school, count(class_cd) as cnt, min(class_std_cnt) as minStd
            from classV
            group by loc, school
            having count(class_cd) > 2 and school is not null
        )
''').show()
# 즉, Urban지역의 ZOWMK 학교는 한반의 학생수 27명이 가장 작은데, 다른 지역의 학교들의 학생수가 가장 작은 반은 27명 이하임

+---------+
|maxMinStd|
+---------+
|       27|
+---------+



### df의 메서드 활용

In [114]:
# Urban 지역의 학교 VVTVA는 반수가 4개 반 학생수 중 가장 작은 수는 25
cdf.select('*').where((cdf.school=='VVTVA') & (cdf.loc=='Urban')).show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     7BL| VVTVA|           29|Urban|     Public|     Standard|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     TB5| VVTVA|           25|Urban|     Public|     Standard|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



In [116]:
# 1. 지역에 따른 학교로 분류하고 학교의 class_cd가 2개 초과인 학교들을 추출
cdf.groupby('loc','school')\
    .agg(count('class_cd').alias('cnt_cd'))\
    .where((col('cnt_cd')>2) & (col('school').isNotNull()))\
    .show()        

+--------+------+------+
|     loc|school|cnt_cd|
+--------+------+------+
|   Rural| VHDHF|     3|
|   Urban| GOOBU|     6|
|Suburban| ZMNYA|     3|
|   Urban| ZOWMK|     4|
|Suburban| UUUQX|     5|
|Suburban| DNQDD|     5|
|   Urban| CUQAM|     4|
|   Urban| IDGFP|     5|
|   Rural| GOKXL|     3|
|   Rural| KZKKE|     5|
|   Rural| VKWQH|     5|
|Suburban| CCAAW|     6|
|   Rural| LAYPA|     3|
|   Urban| QOQTS|     6|
|Suburban| UAGPU|     4|
|   Rural| FBUMG|     3|
|   Urban| VVTVA|     4|
|Suburban| UKPGS|     6|
|Suburban| GJJHK|     5|
|   Rural| OJOBU|     4|
+--------+------+------+
only showing top 20 rows



In [118]:
# 2. 추출된 학교들에서 학생수가 가장 작은 반의 학생수 추출
# 지역에따른 학교의 반들에 대해 가장 학생수가 작은반의 학생수
cdf.groupby('loc','school')\
    .agg(count('class_cd').alias('cntCd'),min('class_std_cnt').alias('minStd'))\
    .where((col('cntCd')>2) & (col('school').isNotNull()))\
    .show()     

+--------+------+-----+------+
|     loc|school|cntCd|minStd|
+--------+------+-----+------+
|   Rural| FBUMG|    3|    14|
|   Rural| GOKXL|    3|    19|
|   Rural| KZKKE|    5|    20|
|   Rural| LAYPA|    3|    17|
|   Rural| OJOBU|    4|    17|
|   Rural| VHDHF|    3|    15|
|   Rural| VKWQH|    5|    18|
|Suburban| CCAAW|    6|    15|
|Suburban| DNQDD|    5|    20|
|Suburban| GJJHK|    5|    21|
|Suburban| UAGPU|    4|    21|
|Suburban| UKPGS|    6|    18|
|Suburban| UUUQX|    5|    15|
|Suburban| ZMNYA|    3|    22|
|   Urban| CIMBB|    4|    17|
|   Urban| CUQAM|    4|    24|
|   Urban| GOOBU|    6|    24|
|   Urban| IDGFP|    5|    17|
|   Urban| QOQTS|    6|    22|
|   Urban| VVTVA|    4|    25|
+--------+------+-----+------+
only showing top 20 rows



In [121]:
# 3. 2번에서 추출된 학생수들 중 가장 많은 수 추출
# # 지역에따른 학교의 반들에 대해 가장 학생수가 작은반의 학생수에서 가장 큰 수 
# 즉, Urban지역의 ZOWMK 학교는 한반의 학생수 27명이 가장 작은데, 다른 지역의 학교들의 학생수가 가장 작은 반은 27명보다 작다
cdf.groupby('loc','school')\
    .agg(count('class_cd').alias('cntCd'),min('class_std_cnt').alias('minStd'))\
    .where((col('cntCd')>2) & (col('school').isNotNull()))\
    .select(max(col('minStd')).alias('maxMinStd'))\
    .show()   

+---------+
|maxMinStd|
+---------+
|       27|
+---------+



#### case3. 지역에 따른 학교로 분류하고 분류된 학교의 class_cd가 2개 초과인 학교에서 학교별 가장 작은 학생수들을 추출 그 중에서 가장 큰 수를 구하시오(27명)
- case2번에서 진행
- 구한 수보다 학생수가 더 많은 반코드와 학생수를 cdf 전체 데이터에서 추출하시오

#### sql 쿼리

In [124]:
spark.sql('''
    select class_cd as class, class_std_cnt as student
    from classV
    where class_std_cnt > (
        select max(minStd) as maxMinStd
        from (
            select loc, school, count(class_cd) as cnt, min(class_std_cnt) as minStd
            from classV
            group by loc, school
            having count(class_cd) > 2 and school is not null
        )
    )
''').show()
spark.sql('select count(*) from classV').show()
# Urban지역의 ZOWMK 학교의 학생수가 가장 작은반보다 학생수가 많은 반은  전체 데이터 102개의 class 중 13 클래스이다

+-----+-------+
|class|student|
+-----+-------+
|  1Q1|     28|
|  OMI|     28|
|  ROP|     28|
|  18K|     31|
|  HKF|     28|
|  0N7|     28|
|  SUR|     28|
|  7BL|     29|
|  A93|     30|
|  YTB|     30|
|  Q0E|     30|
|  QA2|     30|
|  ZBH|     30|
+-----+-------+

+--------+
|count(1)|
+--------+
|     102|
+--------+



#### df 메서드

In [126]:
cdf.select('class_cd', 'class_std_cnt')\
    .where(cdf.class_std_cnt > (
            cdf.groupby('loc','school')\
                .agg(count('class_cd').alias('cntCd'),min('class_std_cnt').alias('minStd'))\
                .where((col('cntCd')>2) & (col('school').isNotNull()))\
                .select(max(col('minStd')).alias('maxMinStd'))\
                .collect()[0][0])
    )\
    .show()           
# Urban지역의 ZOWMK 학교의 학생수가 가장 작은반보다 학생수가 많은 반은 전체 데이터 102개의 class 중 13 클래스이다

+--------+-------------+
|class_cd|class_std_cnt|
+--------+-------------+
|     1Q1|           28|
|     OMI|           28|
|     ROP|           28|
|     18K|           31|
|     HKF|           28|
|     0N7|           28|
|     SUR|           28|
|     7BL|           29|
|     A93|           30|
|     YTB|           30|
|     Q0E|           30|
|     QA2|           30|
|     ZBH|           30|
+--------+-------------+



##### case 4. 시골지역의 사립학교중 표준교육을 진행하는 학교 학급의 평균 학생수보다 학생수가 더 많은 도시 지역의 공립학교면서 특수교육을 진행하는 학교의 모든 정보를 추출하시오

In [40]:
cdf.columns
# loc : Urban, subUrban, Rural
# school_type : Public, Non-public
# teachin_type : Standard, Experimental

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']

### sql query

In [132]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교들의 평균 학생수
spark.sql('''
    select avg(class_std_cnt)
    from classV
    where loc=='Rural' and school_type=='Non-public' and teaching_type=='Standard'
''').show()

+------------------+
|avg(class_std_cnt)|
+------------------+
|              17.0|
+------------------+



In [135]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교 학급의 평균 학생수보다 학생수가 더 많은 도시 지역의 공립학교면서 특수교육을 진행하는 학교의 모든 정보를 추출하시오
spark.sql('''
    select *
    from classV
    where loc == 'Urban' and
          school_type == 'Public' and
          tyaching_type == 'Experimental' and
          class_std_cnt > (
                select avg(class_std_cnt)
                from classV
                where loc=='Rural' and school_type=='Non-public' and teaching_type=='Standard'
          )
''')

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     X6Z| CUQAM|           24|Urban|     Public| Experimental|
|     W8A| GOOBU|           26|Urban|     Public| Experimental|
|     0N7| QOQTS|           28|Urban|     Public| Experimental|
|     X2O| QOQTS|           25|Urban|     Public| Experimental|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+



### df method

In [141]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교 학급의 평균 학생수
cdf.where((cdf.loc=='Rural') &
           (cdf.school_type=='Non-public') &
           (cdf.teaching_type=='Standard'))\
    .select(avg('class_std_cnt'))\
    .collect()[0][0]

17.0

In [143]:
# 시골지역의 사립학교중 표준교육을 진행하는 학교 학급의 평균 학생수보다 학생수가 더 많은 도시 지역의 공립학교면서 특수교육을 진행하는 학교의 모든 정보를 추출하시오
cdf.select('*')\
    .where((cdf.loc=='Urban') &
           (cdf.school_type=='Public') &
           (cdf.teaching_type=='Experimental') &
           (cdf.class_std_cnt > (
                    cdf.where((cdf.loc=='Rural') &
                               (cdf.school_type=='Non-public') &
                               (cdf.teaching_type=='Standard'))\
                        .select(avg('class_std_cnt'))\
                        .collect()[0][0]
           )))\
           .show()

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     X6Z| CUQAM|           24|Urban|     Public| Experimental|
|     W8A| GOOBU|           26|Urban|     Public| Experimental|
|     0N7| QOQTS|           28|Urban|     Public| Experimental|
|     X2O| QOQTS|           25|Urban|     Public| Experimental|
|     A93| VVTVA|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+

